# Nonlinear time-dependent PDEs in FEniCS

We can consider the **time-dependent heat equation** with a forcing term $f(u)$ that is **nonlinear** in the unknown $u$. The equations read as follows:

$$
\frac{\partial u}{\partial t} + u_{xx} = f(u)  \qquad \text{ for } x \in \Omega \text{ and for } t \in [0, T], \\
u = g  \qquad \text{ for } x \in \partial \Omega \text{ and } t \in [0, T], \\
u = u^0 \qquad \text{ for } x \in \Omega \text{ and } t = 0.
$$
$\Omega$ denotes the spatial domain and $\partial \Omega$ the domain boundary. For example for $\Omega = [-L, L]$, the boundary $\partial \Omega$ consists of the two points $-L$ and $L$. We start the simulation at time $t=0$ until the final time $T$.

The solution $u = u(x, t)$, the right-hand side $f = f (u(x, t))$ can be any nonlinear expression function of $u,$ and the boundary value $g = g(x, t)$ may vary in space $(x)$ and time (t). The initial value $u_0$ is a function of space only.


# Newton's method can be used to linearize and solve the equation

In [5]:
from IPython.display import HTML
HTML('<a title="By Ralf Pfeifer (de:Image:NewtonIteration Ani.gif) [GFDL (http://www.gnu.org/copyleft/fdl.html) or CC-BY-SA-3.0 (http://creativecommons.org/licenses/by-sa/3.0/)], via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File%3ANewtonIteration_Ani.gif"><img width="512" alt="NewtonIteration Ani" src="https://upload.wikimedia.org/wikipedia/commons/e/e0/NewtonIteration_Ani.gif"/></a>')

At each time step we have the semidiscretized equation to solve:

$$F(x) = u^n − \Delta t u^n_{xx} - u^{n-1} - \Delta t f^n = 0,$$
for $n=1,\dots, N.$

Since $f(u)$ is a non linear expression of $u$ we need to define a method to linearize the equation. For example, we can use the **Newton's method**.
Newton's method is an iterative method. If we call $k$ the number of iterations, we can summarize the method's steps as follows:

For $k==0$: assign a suitable initial guess for $u^n.$

While $k < k_{max}$

1. Solve $ u^n_k = u^n_{k-1}-\frac{F(u^n_{k-1})}{F'(u^n_{k-1})}$
2. if $F(u^n_k) \: \text{or} \: \|u^n_k - u^n_{k-1}\| < \text{toll}$:
    3. break
3. else:
    4. $k +=1$
    5. Repeat 1. and 2.  
    
The scheme converges when the criteria 2. is satisfied. If this does not happen and the maximum number of iteration is reached the scheme did not converge.

Newton's method converges with second order accuracy and only if the initial guess is "close enough" to the solution.
        

## Some implementation tips

If you use FEniCS you do not need to implement a Newton's scheme since it is already implemented!

Anyway, there are few differences compared to the linear case in the syntax.

In [ ]:
%matplotlib inline
from fenics import *
import pylab
parameters['plotting_backend'] = 'matplotlib'

def f(u):
    return u*u

mesh = UnitIntervalMesh(100)
V = FunctionSpace(mesh, "CG", 1)
u = Function(V)
u0 = Function(V)
v = TestFunction(V)
bc = [DirichletBC(V, 0.0, "x[0]==0.0 && on_boundary"),
      DirichletBC(V, 1.0, "x[0]==1.0 && on_boundary")]

a = u/dt*v*dx + inner(grad(u), grad(v))*dx
L = u0/dt*v*dx + f(u)*v*dx

F = a-L

In this case, we do not define a Trial Function but only a Function. Both the left and the right hand side change at every timestep. Therefore, it is not possible to assemble the matrix before the time loop unlike the linear case. In addition, the form needs to be written such that everything is on the left hand side (F==0) 

In [ ]:
while t<=T:
    t +=dt
    solve(F==0, u, bc)
    u0.assign(u)
    plot(u)
    interactive()

## FEniCS challenge 2
Adapt the solution of FEniCS challenge 1 to solve the bistable cable equation with a cubic f:
* $f(u) = Au(1-u)(u-\alpha)$,
* $\alpha = 0.1$,
* $A = 1.0$.

The other parameters are as before.

